# Classification
In this notebook we classify the CT/PET scans of lung cancer cases by tumor type. 
We are doing this as a baseline task to validate our data loading pipeline.
Most code is taken from [this tutorial](https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html)

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
from torch.utils.data import random_split
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import copy
from thermostability.thermo_dataset import ThermostabilityDataset

cudnn.benchmark = True
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

if torch.cuda.is_available():
    torch.cuda.empty_cache() 
    
cpu = torch.device("cpu")

torch.cuda.list_gpu_processes()

'pynvml module not found, please install pynvml'

# Defining datasets (train/validation) 

In [2]:
trainSet = ThermostabilityDataset("data/train_sequences.fasta")
valSet = ThermostabilityDataset("data/eval_sequences.fasta")

dataloaders = {
    "train": torch.utils.data.DataLoader(trainSet, batch_size=1, shuffle=True, num_workers=4),
    "val": torch.utils.data.DataLoader(valSet, batch_size=1, shuffle=True, num_workers=4)
}

dataset_sizes = {"train": len(trainSet),"val": len(valSet)}



Loading data from cache file:  data/train_sequences.fasta_cache.p
Loading data from cache file:  data/eval_sequences.fasta_cache.p


In [3]:
print(next(enumerate(dataloaders["train"])))

(0, [('MRICFLLLAFLVAETFANELTRCCAGGTRHFKNSNTCSSIKSEGTSMTCQRAASICCLRSLLDNACDSGTDIAKEEESCPSNINILGGGLKKECCDCCLLAKDLLNRNEPCVAPVGFSAGCLRSFNKCCNGDIEITHASEIITGRPLNDPHVLHLGDRCASSHCEHLCHDRGGEKVECSCRSGFDLAPDGMACVDIDECATLMDDCLESQRCLNTPGSFKCIRTLSCGTGYAMDSETERCRDVDECNLGSHDCGPLYQCRNTQGSYRCDAKKCGDGELQNPMTGECTSITCPNGYYPKNGMCNDIDECVTGHNCGAGEECVNTPGSFRCQQKGNLCAHGYEVNGATGFCEDVNECTTGIAACEQKCVNIPGSYQCICDRGFALGPDGTKCEDIDECSIWAGSGNDLCMGGCINTKGSYLCQCPPGYKIQPDGRTCVDVDECAMGECAGSDKVCVNTLGSFKCHSIDCPTNYIHDSLNKNRCNRQPSACGLPEECSKVPLFLTYQFISLARAVPISSHRPAITLFKVSAPNHADTEVNFELQLKTTIVGAPNVLPAIRANFLLQKGEKRNSAVVTLRDSLDGPQTVKLQLLLRMSKKGKNFNTYAANLIVDVAAHKRHNTVHPPLMKIR',), tensor([[43.8922]])])


# Defining model 

In [4]:
from thermostability.hotinfer import HotInfer
model = HotInfer()
model.esmfold.requires_grad_(False)

model.to(device)

HotInfer(
  (esmfold): ESMFold(
    (esm): ESM2(
      (embed_tokens): Embedding(33, 2560, padding_idx=1)
      (layers): ModuleList(
        (0): TransformerLayer(
          (self_attn): MultiheadAttention(
            (k_proj): Linear(in_features=2560, out_features=2560, bias=True)
            (v_proj): Linear(in_features=2560, out_features=2560, bias=True)
            (q_proj): Linear(in_features=2560, out_features=2560, bias=True)
            (out_proj): Linear(in_features=2560, out_features=2560, bias=True)
            (rot_emb): RotaryEmbedding()
          )
          (self_attn_layer_norm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=2560, out_features=10240, bias=True)
          (fc2): Linear(in_features=10240, out_features=2560, bias=True)
          (final_layer_norm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        )
        (1): TransformerLayer(
          (self_attn): MultiheadAttention(
            (k_proj): Lin

# Setup training

In [5]:
from tqdm.notebook import tqdm
import sys

def train_model(model, criterion,optimizer , scheduler, num_epochs=25):
    since = time.time()

    #best_model_wts = copy.deepcopy(model.state_dict())

    best_epoch_loss = sys.float_info.max
    losses = []
    batchEnumeration = []
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
         

            # Iterate over data.
            for idx, (inputs, labels) in enumerate(dataloaders[phase]):
                #inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    
                    print("Outputs", outputs)
                    print("Labels", labels)
                    loss = criterion(outputs,labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                batch_size = len(inputs)
                batch_loss = loss.item() * batch_size
                losses.append(batch_loss)
                batchEnumeration.append(batchEnumeration[-1]+1 if len(batchEnumeration)>0 else 0)

                running_loss += batch_loss
               
            
                if idx % 10 == 0:
                    batch_size = len(inputs)
               
                    tqdm.write("Epoch: [{}/{}], Batch: [{}/{}], loss: {:.6f}".format(
                        epoch,
                        num_epochs,
                        idx + 1,
                        len(dataloaders[phase]),
                        batch_loss / float(batch_size)
                        ), end="\r")
                    
                    
                    
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]


            print(f'{phase} Loss: {epoch_loss:.4f}')

            # deep copy the model
            if phase == 'val' and epoch_loss < best_epoch_loss:
                best_epoch_loss = epoch_loss
                #best_model_wts = copy.deepcopy(model.state_dict())

        print()


    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_epoch_loss:4f}')
    # load best model weights
    #model.load_state_dict(best_model_wts)
    return model

# Define training parameters

In [6]:
criterion = nn.MSELoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

# Run training

In [7]:
model = train_model(model, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=1)

Epoch 0/0
----------
Outputs tensor([[0.2047]], device='cuda:0', grad_fn=<AddmmBackward0>)
Labels tensor([[49.0448]], device='cuda:0')
Outputs tensor([[0.4303]], device='cuda:0', grad_fn=<AddmmBackward0>)
Labels tensor([[41.9732]], device='cuda:0')
Outputs tensor([[0.9685]], device='cuda:0', grad_fn=<AddmmBackward0>)
Labels tensor([[45.6447]], device='cuda:0')


RuntimeError: CUDA out of memory. Tried to allocate 21.05 GiB (GPU 0; 44.43 GiB total capacity; 34.71 GiB already allocated; 5.77 GiB free; 37.52 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

# Evaluation

In [ ]:
def predictDiffs(set="val"):
    with torch.no_grad():
        for inputs, labels in dataloaders[set]:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)

            print("Outputs shape: ", outputs.size())
            print("Labels shape: ", labels.size())

predictDiffs()

diffs = np.array([0, 0.1, 0.2,-0.2, -0.8, 0.1])
plt.hist(diffs, 10)
